In [2]:
from google.cloud import bigquery
from google.cloud.exceptions import NotFound
from sqlalchemy import create_engine
import pandas as pd
import os
import json
import mysql
import mysql.connector as connection
import sqlalchemy


def get_engine():
    db_config = mysql.connector.connect(
        user=os.getenv('DB_USER'),
        password=os.getenv('DB_PASSWORD'),
        host=os.getenv('DB_HOST'),
        database=os.getenv('DB_DATABASE')
    )
    
    return db_config

In [3]:
def list_mysql_tables(request):
    headers = {
        'Access-Control-Allow-Origin': '*',
        'Content-Type': 'application/json'
    }
    engine = get_engine()
    query = '''
        SELECT
            table_schema
            , table_name
            , table_rows
        FROM information_schema.tables
        WHERE table_schema = "github" AND TABLE_TYPE = "BASE TABLE"
    '''

    result_dataframe = pd.read_sql(query, engine)
    engine.close()
    return (result_dataframe.to_json(orient='records'), 200)

In [4]:
class RequestArgsMock:
    def __init__(self, **kwargs):
        self.args = kwargs

request = RequestArgsMock(cliente='128')
list_mysql_tables(request)

/var/tmp/ipykernel_7200/3391906181.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_dataframe = pd.read_sql(query, engine)


('[{"TABLE_SCHEMA":"github","TABLE_NAME":"customers","TABLE_ROWS":122},{"TABLE_SCHEMA":"github","TABLE_NAME":"employees","TABLE_ROWS":23},{"TABLE_SCHEMA":"github","TABLE_NAME":"offices","TABLE_ROWS":7},{"TABLE_SCHEMA":"github","TABLE_NAME":"orderdetails","TABLE_ROWS":2996},{"TABLE_SCHEMA":"github","TABLE_NAME":"orders","TABLE_ROWS":326},{"TABLE_SCHEMA":"github","TABLE_NAME":"payments","TABLE_ROWS":273},{"TABLE_SCHEMA":"github","TABLE_NAME":"productlines","TABLE_ROWS":7},{"TABLE_SCHEMA":"github","TABLE_NAME":"products","TABLE_ROWS":110},{"TABLE_SCHEMA":"github","TABLE_NAME":"SalesTable","TABLE_ROWS":2850}]',
 200)

In [5]:
def copy_table_mysql_to_bigquery(request):
    table_source = request.args['table_source']
    table_destination = request.args['table_destination']
    project_id = os.getenv('PROJECT_ID')
    schema = os.getenv('SCHEMA')
    table_id =  '{0}.{1}.{2}'.format(project_id, schema, table_destination)

    headers = {
      'Access-Control-Allow-Origin': '*',
      'Content-Type': 'application/json'
    }

    if table_source.strip() == "" or table_destination.strip() == "":
        return json.dumps({'result': 'error', 'description': 'no table name'})

    engine = get_engine()
    query = 'SELECT * FROM {0}'.format(table_source)
    result_dataframe = pd.read_sql(query, engine)

    client = bigquery.Client()

    job_config = bigquery.LoadJobConfig(
        write_disposition="WRITE_TRUNCATE",
    )

    job = client.load_table_from_dataframe(
        result_dataframe, table_id, job_config=job_config
    )
    job.result()

    try:
        client.get_table(table_id)
    except NotFound:
        return json.dumps({'result': 'error', 'description': 'table not found'})

    return json.dumps({'result': 'ok'})

In [6]:
class RequestArgsMock:
    def __init__(self, **kwargs):
        self.args = kwargs

request = RequestArgsMock(table_source='employees', table_destination='employees')
copy_table_mysql_to_bigquery(request)

/var/tmp/ipykernel_7200/3893684079.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_dataframe = pd.read_sql(query, engine)


'{"result": "ok"}'

In [7]:
def get_rows_from_bigquery(request):
    table_name = request.args['table_name']
    project_id = 'chatmine-388722'
    schema = 'co_dtkl'
    table_id =  '{0}.{1}.{2}'.format(project_id, schema, table_name)

    headers = {
      'Access-Control-Allow-Origin': '*',
      'Content-Type': 'application/json'
    }

    if table_name.strip() == "":
        return json.dumps({'result': 'error', 'description': 'no table name'})

    client = bigquery.Client()

    try:
        client.get_table(table_id)
    except NotFound:
        return json.dumps({'result': 'error', 'description': 'table not found'})

    query = 'SELECT COUNT(*) filas FROM `{0}`.`{1}`'.format(schema, table_name)
    query_job = client.query(query)
    query_job.result()

    rows = list(query_job)
    return json.dumps({'result': 'ok', 'rows': rows[0].filas})

In [8]:
class RequestArgsMock:
    def __init__(self, **kwargs):
        self.args = kwargs

request = RequestArgsMock(table_name='employees')
get_rows_from_bigquery(request)

'{"result": "ok", "rows": 23}'